# Inverse Kinematics and PID control
In this homework assignemnt, you will be tasked with creating an IK solver and controlling a robot arm using PID control (specifically the Franka Panda 7DoF robot). We will be creating two versions of the controller, one that uses pure velocity control and another that using torque control with feedforward dynamics. 

### Import mujoco and numpy

In [1]:
import mujoco
import mujoco_viewer
import numpy as np
import scipy as sp

from autograder import autograder


### Franka Emika Panda Robot
We will be importing the Franka Panda robot. Take a look at the robot tree in `franka_emika_panda/panda.xml` file. Rather than importing the robot, we will instead import a specific `scene_X.xml` file that specifies different joint actuation on the robot. Note that the `panda.xml` only specifies the robot body and joints. There are two other files that specify the type of actuation on the robot (`vel_act.xml` and `torque_act.xml`) which we will use to demonstrate the different levels of robot control. 

In [2]:
def hat(w):
    return np.array([
        [0., -w[2], w[1]],
        [w[2], 0., -w[0]],
        [-w[1], w[0], 0.]])

def unhat(R):
    return np.array([R[2,1], R[0,2], R[1,0]])


In [3]:
autograder.testQ1(hat, unhat)

Passed Debug test


In [4]:

def getSitePose(model, data, site_name):
    site_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SITE, site_name)
    p = data.site_xpos[site_id]
    R = data.site_xmat[site_id].reshape((3,3))
    return (R, p)

def getSiteJacobian(model, data, body_name):
    body_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SITE, body_name)
    jacp = np.zeros((3, model.nv))
    jacr = np.zeros((3, model.nv))
    mujoco.mj_jacSite(model, data, jacp, jacr, body_id)
    return np.concatenate([jacp, jacr], axis=0)

def calcIKStep(jac, x_des, x_curr):
    dp = x_des[1] - x_curr[1]
    dw = unhat(sp.linalg.logm(x_des[0]@x_curr[0].T))
    res = np.concatenate([dp, dw])
    return np.linalg.pinv(jac) @ res

def IKSolve(q, jac, x_des, x_curr):
    dq = calcIKStep(jac, x_des, x_curr)
    return q + dq

In [5]:
model   = mujoco.MjModel.from_xml_path('./franka_emika_panda/scene_torque_actuators.xml')
data    = mujoco.MjData(model)

def simulateCtrl(ctrl, render=True, t_max=1000):
    data.qpos = model.key_qpos
    mujoco.mj_step(model, data)
    if render:
        viewer = mujoco_viewer.MujocoViewer(model, data)
        viewer.vopt.frame = 3
    for t in range(t_max):
        u = ctrl(model, data)
        data.ctrl = u 
        mujoco.mj_step(model, data)
        if render:
            if viewer.is_alive:
                viewer.render()
    if render:
        viewer.close()
        

## Visualization
The following code will create a openGL window which will render the environment to you. However, just running it will not do anything. As part of the code, you will have to run `viewer.render()` each time after you step the simulation forward. Depending on how often you run the render function, the visualization will appear faster/slower than real-time. The true time stepping is defined in the `.xml` file. 

In [6]:
def ctrl(model, data):
    # PD control parameters
    kp = 350.
    kd = 50.
    # simulate and render
    ee_pose     = getSitePose(model, data, "ee_site")
    site_pose   = getSitePose(model, data, "target_site")
    jac         = getSiteJacobian(model, data, "ee_site")
    # dq          = calcIKStep(jac, site_pose, ee_pose)
    q = data.qpos
    qvel = data.qvel
    q_des = IKSolve(q, jac, site_pose, ee_pose)
    qddot_des = kp * (q_des - q) + kd * (-qvel)
    mass_matrix = np.zeros((model.nv, model.nv))
    mujoco.mj_fullM(model, mass_matrix, data.qM)
    tau = mass_matrix@qddot_des + data.qfrc_bias
    return tau

In [7]:
simulateCtrl(ctrl)

/home/anon/miniconda3/envs/meng443arm/lib/python3.10/site-packages/glfw/__init__.py:906: GLFWError: (65544) b'Wayland: Standard cursor shape unavailable'
  warnings.warn(message, GLFWError)


Pressed ESC
Quitting.


In [17]:
mass_matrix

array([[ 1.50865254e+00, -3.61426911e-02,  1.37131507e+00,
         5.09243427e-04,  9.51639626e-02,  2.98947839e-04,
        -8.49094264e-03],
       [-3.61426911e-02,  2.79452677e+00, -3.53737552e-02,
        -1.26786438e+00, -2.83948065e-02, -9.43958213e-02,
         3.57179342e-04],
       [ 1.37131507e+00, -3.53737552e-02,  1.47131507e+00,
         5.09243427e-04,  9.51639626e-02,  2.98947839e-04,
        -8.49094264e-03],
       [ 5.09243427e-04, -1.26786438e+00,  5.09243427e-04,
         1.05804436e+00,  2.27639164e-02,  1.29921866e-01,
        -2.36806200e-04],
       [ 9.51639626e-02, -2.83948065e-02,  9.51639626e-02,
         2.27639164e-02,  1.44164678e-01, -1.28063032e-03,
        -1.57039620e-03],
       [ 2.98947839e-04, -9.43958213e-02,  2.98947839e-04,
         1.29921866e-01, -1.28063032e-03,  1.53980606e-01,
        -2.05378743e-04],
       [-8.49094264e-03,  3.57179342e-04, -8.49094264e-03,
        -2.36806200e-04, -1.57039620e-03, -2.05378743e-04,
         1.0668415

(7,)

(7,)